# GRPO Fine-tuning for AMR Prescription Validation

This notebook implements Group Relative Policy Optimization (GRPO) for fine-tuning language models on AMR (Antimicrobial Resistance) prescription validation tasks.

**GRPO** is a reinforcement learning method that optimizes language models by:
- Generating multiple outputs per prompt
- Computing rewards for each output using LLM-as-a-Judge
- Using group-relative advantages for stable optimization
- Updating the model to favor high-reward outputs

**Dataset**: ICMR 2025 Antimicrobial Treatment Guidelines
**Task**: Validate prescriptions with step-by-step clinical reasoning


## 1. Setup and Installation


In [ ]:
# Install required packages
# Run this cell first!

import os
import sys

# Check if running in Colab
if "COLAB_" in "".join(os.environ.keys()):
    print("🔧 Installing packages for Google Colab...")
    !pip install -q unsloth bitsandbytes accelerate peft trl transformers datasets
else:
    print("🔧 Installing packages for local environment...")
    !pip install -q unsloth transformers datasets groq pydantic tqdm aiohttp nest-asyncio

print("✅ Installation complete!")


In [ ]:
# Import libraries
import torch
import json
import numpy as np
import asyncio
import aiohttp
import nest_asyncio
import hashlib
import time
from pathlib import Path
from typing import List, Dict, Optional, Tuple, Any
from datasets import Dataset, load_dataset
from unsloth import FastLanguageModel
from tqdm.auto import tqdm
import warnings

warnings.filterwarnings('ignore')
nest_asyncio.apply()

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
print(f"✅ All imports successful!")


## 2. Configuration


In [ ]:
# ============================================================================
# MODEL CONFIGURATION
# ============================================================================

MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"
OUTPUT_DIR = "./grpo_amr_model"

# ============================================================================
# EVALUATION API CONFIGURATION
# ============================================================================

# IMPORTANT: Set your Supabase Edge Function URL here
# After deploying: supabase functions deploy evaluate-prescription
API_BASE_URL = "https://your-project.supabase.co/functions/v1/evaluate-prescription"

# Priority metrics for training (faster, focused on key aspects)
PRIORITY_METRICS = ['clinical_accuracy', 'guideline_adherence', 'reasoning_completeness']

# All metrics for comprehensive evaluation
ALL_METRICS = [
    'clinical_accuracy',
    'guideline_adherence',
    'reasoning_completeness',
    'safety_awareness',
    'decision_appropriateness',
    'reference_accuracy'
]

# Reward weights for each metric (must sum to 1.0)
REWARD_WEIGHTS = {
    'clinical_accuracy': 0.25,
    'guideline_adherence': 0.25,
    'reasoning_completeness': 0.20,
    'safety_awareness': 0.15,
    'decision_appropriateness': 0.10,
    'reference_accuracy': 0.05
}

# API Configuration
API_CONFIG = {
    "max_concurrent_requests": 50,
    "timeout_seconds": 45,
    "max_retries": 3,
    "retry_delay": 2.0,
    "use_cache": True,
}

# Evaluation frequency (evaluate every N training steps)
EVAL_FREQUENCY = 10

# ============================================================================
# GRPO HYPERPARAMETERS
# ============================================================================

GRPO_CONFIG = {
    "num_generations_per_prompt": 2,
    "batch_size": 2,
    "learning_rate": 5e-6,
    "num_train_epochs": 3,
    "max_length": 1024,
    "temperature": 0.7,
    "top_p": 0.95,
    "kl_coef": 0.05,
    "clip_range": 0.2,
    "vf_coef": 0.1,
    "eval_frequency": EVAL_FREQUENCY,
    "use_priority_metrics": True,
}

# ============================================================================
# UNSLOTH CONFIGURATION
# ============================================================================

MAX_SEQ_LENGTH = 2048
LOAD_IN_4BIT = True
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0
LORA_TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]

print("✅ Configuration loaded successfully!")
print(f"📍 API Base URL: {API_BASE_URL}")
print(f"🎯 Using Metrics: {'PRIORITY (3 metrics)' if GRPO_CONFIG['use_priority_metrics'] else 'ALL (6 metrics)'}")
print(f"📊 Evaluation Frequency: Every {EVAL_FREQUENCY} steps")
print(f"⚡ Batch Size: {GRPO_CONFIG['batch_size']}")
print(f"🔢 Generations per prompt: {GRPO_CONFIG['num_generations_per_prompt']}")


## 3. Load Dataset

We'll load the prepared GRPO dataset. If you haven't prepared it yet, run:
```bash
python prepare_grpo_dataset.py
```


In [ ]:
# Load dataset
from datasets import load_from_disk

# Path to prepared dataset
data_dir = Path("./data")
train_dataset_path = data_dir / "train_hf"

print(f"📂 Loading dataset from: {train_dataset_path}")

if not train_dataset_path.exists():
    print("❌ Dataset not found!")
    print("Please run: python prepare_grpo_dataset.py")
    print("This will convert your merged AMR dataset to GRPO format.")
else:
    train_dataset = load_from_disk(str(train_dataset_path))
    print(f"✅ Loaded {len(train_dataset)} training examples")
    
    # Show dataset structure
    print(f"\n📋 Dataset columns: {train_dataset.column_names}")
    
    # Show example
    print(f"\n{'='*80}")
    print("Example Training Instance:")
    print(f"{'='*80}")
    example = train_dataset[0]
    print(f"\n📝 Prompt (first 300 chars):")
    print(example['prompt'][:300] + "...")
    print(f"\n✨ Reference (first 200 chars):")
    print(example['reference'][:200] + "...")
    print(f"\n🏷️  Task Type: {example.get('task_type', 'unknown')}")
    print(f"{'='*80}")


## 4. Load Model and Tokenizer


In [ ]:
# Load model and tokenizer with Unsloth (optimized for speed and memory)
print(f"🔄 Loading model: {MODEL_NAME}")
print(f"This may take a few minutes on first run...")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,  # Auto-detect dtype
    load_in_4bit=LOAD_IN_4BIT,
)

print(f"\n✅ Model and tokenizer loaded successfully!")
print(f"   Model: {MODEL_NAME}")
print(f"   Max sequence length: {MAX_SEQ_LENGTH}")
print(f"   4-bit quantization: {LOAD_IN_4BIT}")
print(f"   Vocab size: {len(tokenizer)}")


## 5. API-Integrated Reward Model

This reward model integrates with your Supabase Edge Function to evaluate model outputs using LLM-as-a-Judge.


In [ ]:
# Add LoRA adapters with Unsloth (super fast!)
print("🔧 Adding LoRA adapters with Unsloth optimization...")

model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_R,
    target_modules=LORA_TARGET_MODULES,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

print("\n✅ LoRA adapters added successfully!")
print("\n📊 Trainable Parameters:")
model.print_trainable_parameters()


In [ ]:
class APIIntegratedRewardModel:
    """
    Reward model that integrates with Supabase Edge Function for evaluation.
    
    Features:
    - Async batch processing with concurrency control
    - Response caching to avoid duplicate API calls
    - Retry logic with exponential backoff
    - Priority metrics mode for faster training
    """
    
    def __init__(
        self,
        api_base_url: str,
        metrics: List[str],
        weights: Dict[str, float],
        api_config: Dict[str, Any],
        use_cache: bool = True
    ):
        self.api_base_url = api_base_url
        self.metrics = metrics
        self.weights = weights
        self.api_config = api_config
        self.use_cache = use_cache
        
        self.cache = {} if use_cache else None
        self.stats = {
            'total_calls': 0,
            'cache_hits': 0,
            'api_errors': 0,
            'total_api_time': 0.0
        }
        
        print(f"✅ Reward Model initialized with {len(metrics)} metrics")
        print(f"   Metrics: {metrics}")
        print(f"   Caching: {'Enabled' if use_cache else 'Disabled'}")
    
    def _create_cache_key(self, context: Dict, model_output: str, ground_truth: str) -> str:
        """Create a unique cache key for an API call."""
        content = f"{json.dumps(context)}|{model_output}|{ground_truth}"
        return hashlib.md5(content.encode()).hexdigest()
    
    async def _call_api_async(
        self,
        session: aiohttp.ClientSession,
        context: Dict,
        model_output: str,
        ground_truth: str,
        retry_count: int = 0
    ) -> Dict[str, Any]:
        """Make async API call to evaluation endpoint."""
        
        # Check cache
        if self.use_cache:
            cache_key = self._create_cache_key(context, model_output, ground_truth)
            if cache_key in self.cache:
                self.stats['cache_hits'] += 1
                return self.cache[cache_key]
        
        # Prepare request
        payload = {
            "patient_case": context,
            "model_output": model_output,
            "ground_truth": ground_truth,
            "metrics": self.metrics
        }
        
        try:
            start_time = time.time()
            
            async with session.post(
                self.api_base_url,
                json=payload,
                timeout=aiohttp.ClientTimeout(total=self.api_config['timeout_seconds'])
            ) as response:
                elapsed = time.time() - start_time
                self.stats['total_api_time'] += elapsed
                self.stats['total_calls'] += 1
                
                if response.status == 200:
                    result = await response.json()
                    
                    # Cache the result
                    if self.use_cache:
                        self.cache[cache_key] = result
                    
                    return result
                else:
                    error_text = await response.text()
                    raise Exception(f"API returned status {response.status}: {error_text}")
        
        except Exception as e:
            # Retry logic
            if retry_count < self.api_config['max_retries']:
                await asyncio.sleep(self.api_config['retry_delay'] * (2 ** retry_count))
                return await self._call_api_async(
                    session, context, model_output, ground_truth, retry_count + 1
                )
            else:
                self.stats['api_errors'] += 1
                # Return default low scores
                return {
                    "success": False,
                    "evaluations": {metric: {"score": 1} for metric in self.metrics},
                    "weighted_reward": 0.0
                }
    
    async def _evaluate_batch_async(
        self,
        contexts: List[Dict],
        generated: List[str],
        references: List[str]
    ) -> List[Dict]:
        """Evaluate a batch of generations using async API calls."""
        semaphore = asyncio.Semaphore(self.api_config['max_concurrent_requests'])
        
        async def evaluate_single(ctx, gen, ref):
            async with semaphore:
                async with aiohttp.ClientSession() as session:
                    return await self._call_api_async(session, ctx, gen, ref)
        
        tasks = [
            evaluate_single(ctx, gen, ref)
            for ctx, gen, ref in zip(contexts, generated, references)
        ]
        results = await asyncio.gather(*tasks)
        
        return results
    
    def evaluate_batch(
        self,
        contexts: List[Dict],
        generated: List[str],
        references: List[str]
    ) -> List[Dict]:
        """Synchronous wrapper for batch evaluation."""
        loop = asyncio.get_event_loop()
        return loop.run_until_complete(
            self._evaluate_batch_async(contexts, generated, references)
        )
    
    def compute_batch_rewards(
        self,
        contexts: List[Dict],
        generated: List[str],
        references: List[str]
    ) -> Tuple[List[float], List[Dict]]:
        """Compute rewards for a batch of generations."""
        results = self.evaluate_batch(contexts, generated, references)
        
        rewards = []
        all_scores = []
        
        for result in results:
            if result.get("success", False):
                reward = result.get("weighted_reward", 0.0)
                scores = result.get("evaluations", {})
            else:
                reward = 0.0
                scores = {}
            
            rewards.append(reward)
            all_scores.append(scores)
        
        return rewards, all_scores
    
    def print_stats(self):
        """Print cache and API usage statistics."""
        print(f"\n{'='*60}")
        print("Reward Model Statistics")
        print(f"{'='*60}")
        print(f"Total API calls: {self.stats['total_calls']}")
        print(f"Cache hits: {self.stats['cache_hits']}")
        if self.stats['total_calls'] > 0:
            cache_rate = (self.stats['cache_hits'] / (self.stats['total_calls'] + self.stats['cache_hits'])) * 100
            print(f"Cache hit rate: {cache_rate:.1f}%")
        print(f"API errors: {self.stats['api_errors']}")
        if self.stats['total_calls'] > 0:
            avg_time = self.stats['total_api_time'] / self.stats['total_calls']
            print(f"Average API call time: {avg_time:.2f}s")
        print(f"{'='*60}\n")


# Initialize reward model
metrics_to_use = PRIORITY_METRICS if GRPO_CONFIG["use_priority_metrics"] else ALL_METRICS

reward_model = APIIntegratedRewardModel(
    api_base_url=API_BASE_URL,
    metrics=metrics_to_use,
    weights=REWARD_WEIGHTS,
    api_config=API_CONFIG,
    use_cache=API_CONFIG["use_cache"]
)

print(f"\n🎯 Using metrics: {metrics_to_use}")
